<a href="https://colab.research.google.com/github/micaelakorol21/water_quality/blob/main/water_quality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [231]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

<h1>Clasificación de Agua Potable</h1>
<h2>🚰El objetivo de este proyecto es predecir si una muestra de agua es potable basándonos en características físico-químicas y ambientales.</h2>

In [ ]:
df = pd.read_csv('https://media.githubusercontent.com/media/micaelakorol21/dataset_water_quality/refs/heads/main/Water_quality.csv')

<p style="font-size:20px;">🔍Exploración de los datos:</p>

In [ ]:
df.shape
# El dataset poseé: 24 columnas y 698.575 filas.

In [ ]:
# Observamos los tipos de datos de cada columna:
df.dtypes

In [ ]:
# Observamos las columnas que tienen datos faltantes:
df.isna().sum()

In [ ]:
# Calculamos el maximo valor faltante:
max_missing = df.isna().sum().max()
print(f"El valor maximo de faltantes es de {max_missing}")

<b style='font-size:22px'>Renombramos las columnas</b>

In [ ]:
df = df.rename(columns={
    'Index': 'Índice',
    'pH': 'pH',
    'Iron': 'Plata',
    'Nitrate': 'Nitrato',
    'Chloride': 'Cloruros',
    'Lead': 'Plomo',
    'Zinc': 'Zinc',
    'Color': 'Color',
    'Turbidity': 'Turbidez',
    'Fluoride': 'Fluoruro',
    'Copper': 'Cobre',
    'Odor': 'Olor',
    'Sulfate': 'Sulfatos',
    'Conductivity': 'Conductividad',
    'Chlorine': 'Cloro residual libre',
    'Manganese': 'Manganeso',
    'Total Dissolved Solids': 'Sólidos disueltos totales',
    'Source': 'Fuente',
    'Water Temperature': 'Temperatura del agua',
    'Air Temperature': 'Temperatura del aire',
    'Month': 'Mes',
    'Day': 'Día',
    'Time of Day': 'Hora del día',
    'Target': 'Objetivo'
})

<b style="font-size:16px">El conjunto de datos incluye 24 columnas, de las cuales 23 son variables predictoras y 1 es la variable objetivo que indica si el agua <b style='color:red'> es potable (1) o no (0)  </b> .</b>

In [ ]:
df.columns

<b style="font-size:22px;"> Análisis de la variable objetivo: Objetivo</b>

In [ ]:
# Conteo de clases
df['Objetivo'].value_counts()

In [ ]:
sns.countplot(data=df, x='Objetivo')
plt.title('Distribución de Agua Potable vs No Potable')
plt.xticks([0, 1], ['No Potable (0)', 'Potable (1)'])
plt.ylabel('Cantidad de muestras')
plt.xlabel('Clase')
plt.show()

<b style='font-size:16px'>El dataset está desbalanceado: hay más muestras de agua no potable (457841) que de agua potable (240734).</b>

<b style="font-size:22px">Análisis de la columna pH </b>

In [ ]:
# Observamos la distribuición de la columna pH:
plt.figure(figsize=(10, 6))
sns.histplot(df['pH'], kde=True)
plt.title('pH')
plt.xlabel('Valor')
plt.ylabel('Frecuencia')
plt.show()

In [ ]:
df['pH'].describe()

In [ ]:
#Eliminar outliers de la fila pH
col = 'pH'
Q1 = df[col].quantile(0.25)
Q3 = df[col].quantile(0.75)
IQR = Q3 - Q1
# Eliminamos las filas que contienen valores en la columna pH que están fuera del rango definido
df = df[(df[col] >= Q1 - 1.5 * IQR) & (df[col] <= Q3 + 1.5 * IQR)]
df['pH'].isna().sum() #Analizo si hay valor nulos, me da 0.
df['pH'].describe()

In [ ]:
plt.figure()
sns.histplot(df['pH'], kde=True)
plt.title('pH')
plt.xlabel('Valor')
plt.ylabel('Frecuencia')
plt.show() # ¿no es mejor usar z score?

<p>Análisis de pH 📝 Después de eliminar los valores nulos y los outliers en la columna pH, se observaron variaciones significativas en las estadísticas descriptivas. Se eliminaron aproximadamente 29,870 observaciones, principalmente debido a valores extremos o faltantes.

En cuanto a la media, hubo un ligero aumento, pasando de 7.45 a 7.46, lo que sugiere que los valores atípicos eliminados estaban en su mayoría por debajo del promedio general. La desviación estándar también se redujo notablemente de 0.85 a 0.73, lo que indica que los datos restantes están ahora más concentrados alrededor de la media, con menos dispersión.

El valor mínimo cambió de 2.06 a 5.36 y el máximo de 12.89 a 9.57, lo que confirma que se eliminaron los valores más extremos en ambos extremos del rango. Asimismo, los percentiles (25%, 50% y 75%) se ajustaron ligeramente, reflejando una distribución más limpia y estable</p>

In [ ]:
df.isna().sum()

<b style="font-size:22px">Análisis de la columna: Plata </b>

In [ ]:
df['Plata'].describe()

In [ ]:
col_p = 'Plata'
q1p = df[col_p].quantile(0.25)
q3p = df[col_p].quantile(0.75)
iqrp = q3p - q1p
df = df[(df[col_p] >= q1p - 1.5 * iqrp) & (df[col_p] <= q3p + 1.5 * iqrp)]
print(df[col_p].describe())

In [ ]:
# Crear figura y ejes
fig, ax = plt.subplots(figsize=(10, 6))

# Histograma normal acotado hasta 0.001
sns.histplot(df['Plata'], kde=True, ax=ax)

# Acotar eje X para observar mejor valores pequeños
ax.set_xlim(0, 0.001)

# Agregar títulos y etiquetas
ax.set_title('Distribución de Plata (Escala normal, hasta 0.001)')
ax.set_xlabel('Valor (¿g/L o mg/L?)')  # Cambiar al confirmar la unidad
ax.set_ylabel('Frecuencia')

# Ajustar presentación
plt.tight_layout()
plt.show()

<b style="font-size:22px">Análisis de la columna: Nitrato </b>

In [ ]:
df['Nitrato'].describe()

In [ ]:
# Observamos la distribuición de la columna Nitrato:

plt.figure(figsize=(10, 6))
sns.histplot(df['Nitrato'], kde=True)
plt.title('Nitrato')
plt.xlabel('Valor')
plt.ylabel('Frecuencia')
plt.show()

In [ ]:
df['Nitrato'].describe()

In [ ]:
#Eliminar outliers de la columna Nitrato

col2 = 'Nitrato'  # ejemplo
Q1 = df[col2].quantile(0.25)
Q3 = df[col2].quantile(0.75)
IQR = Q3 - Q1
df = df[(df[col2] >= Q1 - 1.5 * IQR) & (df[col2] <= Q3 + 1.5 * IQR)]
df['Nitrato'].isna().sum() #Analizo si hay valor nulos, me da 0.
df['Nitrato'].describe()

In [ ]:
plt.figure()
sns.histplot(df['Nitrato'], kde=True)
plt.title('Nitrato')
plt.xlabel('Valor')
plt.ylabel('Frecuencia')
plt.show()

<p>Análisis de Nitrato 📝 Después de eliminar los valores nulos y los outliers, se eliminaron aproximadamente 17,370 observaciones, principalmente debido a valores extremos o faltantes.

En cuanto a la media, pasó de 6.13 a 5.76, lo que sugiere que los valores atípicos eliminados estaban en su mayoría por arriba del promedio general. La desviación estándar también se redujo de 3.21 a 2.28, lo que indica que los datos restantes están ahora más concentrados alrededor de la media, con menos dispersión.

El valor mínimo no cambió y el máximo paso de 73.07 a 13.14, lo que confirma que se eliminaron los valores más extremos en el extremo superior del rango. Asimismo, los percentiles (25%, 50% y 75%) se ajustaron ligeramente, reflejando una distribución más limpia y estable

El limite del código alimentario argentino para nitratos es de 45 mg/l, hay que evaluar que unidad maneja el dataset para coincidir con el promedio aprox.</p>

<b style="font-size:22px">Análisis de la columna: Cloruros</b>

In [ ]:
# Observamos la distribuición de la columna Cloruros:
plt.figure(figsize=(10, 6))
sns.histplot(df['Cloruros'], kde=True)
plt.title('Cloruros')
plt.xlabel('Valor')
plt.ylabel('Frecuencia')
plt.show()

In [ ]:
df['Cloruros'].describe()

In [ ]:
#Eliminar outliers de la fila Cloruros

col3 = 'Cloruros'  # ejemplo
Q1 = df[col3].quantile(0.25)
Q3 = df[col3].quantile(0.75)
IQR = Q3 - Q1
df = df[(df[col3] >= Q1 - 1.5 * IQR) & (df[col3] <= Q3 + 1.5 * IQR)]
df['Cloruros'].isna().sum() #Analizo si hay valor nulos, me da 0.
df['Cloruros'].describe()

In [ ]:
plt.figure()
sns.histplot(df['Cloruros'], kde=True)
plt.title('Cloruros')
plt.xlabel('Valor')
plt.ylabel('Frecuencia')
plt.show()

<p>Análisis de Cloruros 📝 Después de eliminar los valores nulos y los outliers, se eliminaron 15,717 observaciones, principalmente debido a valores extremos o faltantes.

En cuanto a la media, pasó de 182.68 a 175.57, lo que sugiere que los valores atípicos eliminados estaban en su mayoría por arriba del promedio general. La desviación estándar también se redujo de 66.58 a 52.55, lo que indica que los datos restantes están ahora más concentrados alrededor de la media, con menos dispersión.

El valor mínimo no cambió y el máximo paso de 1321 a 334, lo que confirma que se eliminaron los valores más extremos en el extremo superior del rango. Asimismo, los percentiles (25%, 50% y 75%) se ajustaron ligeramente, reflejando una distribución más limpia y estable

El limite del código alimentario argentino para nitratos es de 200 mg/l, se considera que utiliza esta unidad ya que tiene coherencia.</p>

<b style="font-size:22px">Análisis de la columna: Zinc</b>

In [ ]:
# Observamos la distribuición de la columna Zinc:
plt.figure(figsize=(10, 6))
sns.histplot(df['Zinc'], kde=True)
plt.title('Zinc')
plt.xlabel('Valor')
plt.ylabel('Frecuencia')
plt.show()

In [ ]:
df['Zinc'].describe()

In [ ]:
#Eliminar outliers de la fila Zinc

col5 = 'Zinc'  # ejemplo
Q1 = df[col5].quantile(0.25)
Q3 = df[col5].quantile(0.75)
IQR = Q3 - Q1
df = df[(df[col5] >= Q1 - 2.5 * IQR) & (df[col5] <= Q3 + 2.5 * IQR)]
df['Zinc'].isna().sum() #Analizo si hay valor nulos, me da 0.
df['Zinc'].describe()

In [ ]:
#Volvemos a graficar
plt.figure()
sns.histplot(df['Zinc'], kde=True)
plt.title('Zinc')
plt.xlabel('Valor')
plt.ylabel('Frecuencia')
plt.show()

<p>Análisis de Zinc 📝 Después de eliminar los valores nulos y los outliers, se eliminaron solo 4,594 observaciones, principalmente debido a valores extremos o faltantes.

En cuanto a la media, pasó de 1.53 a 1.46, lo que sugiere que los valores atípicos eliminados estaban en su mayoría por arriba del promedio general. La desviación estándar también se redujo de 1.50 a 1.42, lo que indica que los datos restantes están ahora más concentrados alrededor de la media, con menos dispersión.

El valor mínimo no cambió y el máximo paso de 28.36 a 6.72, lo que confirma que se eliminaron los valores más extremos en el extremo superior del rango. Asimismo, los percentiles (25%, 50% y 75%) se ajustaron ligeramente, reflejando una distribución más limpia y estable

El limite del código alimentario argentino para nitratos es de 5 mg/l, se considera que utiliza esta unidad ya que tiene coherencia.</p>

<b style='font-size:22px'> Análisis de la columna: Plomo </b>

In [ ]:
df['Plomo'].describe()

<p>La variable "Plomo" contiene principalmente números muy pequeños, con un promedio de 0.0013 y una desviación estándar de 0.0305. Dado que los valores son tan bajos, aplicamos una transformación logarítmica para comprimir los valores y mejorar la distribución, facilitando su análisis y modelado posterior.</p>

In [ ]:
df['Plomo'] = df['Plomo'] + 1e-10  # evitar log(0)

# Aplicar el logaritmo natural
df['Plomo'] = np.log(df['Plomo'])

In [ ]:
# Eliminamos outliers:

col_p = 'Plomo'
q1p = df[col_p].quantile(0.25)
q3p = df[col_p].quantile(0.75)
iqrp = q3p - q1p
df = df[(df[col_p] >= q1p - 1.5 * iqrp) & (df[col_p] <= q3p + 1.5 * iqrp)]
#Analizo si quedó algún valor nulo:
df['Plomo'].isna().sum()
print(df[col_p].describe())

In [ ]:
# Visualización con histograma:
plt.figure(figsize=(8, 6))
sns.histplot(df['Plomo'], kde=True, bins=50)
plt.title('Distribución de Plomo después de aplicar Logaritmo Natural')
plt.xlabel('Valor de Plomo (log)')
plt.ylabel('Frecuencia')
plt.show()

<p>La transformación logarítmica expandió la escala de los valores pequeños, mostrando la alta concentración de valores de "Plomo" cercanos a cero en un pico negativo. A pesar de esto, la distribución sigue siendo significativamente no normal, con la mayoría de los datos agrupados en valores muy bajos.
</p>

<b style='font-size:22px'> Análisis de la columna: Manganeso </b>

In [ ]:
df['Manganeso'].describe()

<p> La variable "Manganeso" tiene un promedio de 0.091, con una gran variabilidad (desviación estándar de 0.432). La mayoría de los valores son muy pequeños, con un mínimo de 1.38e-46, pero hay algunos valores altos, como 15.26, lo que sugiere la presencia de valores extremos.
<br>
Aplicaré el logaritmo natural para reducir la variabilidad y hacer la distribución más estable, facilitando el análisis.
</p>

In [ ]:
df['Manganeso'] = df['Manganeso'] + 1e-10  # evitar log(0)
# Aplicar el logaritmo natural
df['Manganeso'] = np.log(df['Manganeso'])

In [ ]:
# Eliminamos los Outliers:
col_p = 'Manganeso'
q1p = df[col_p].quantile(0.25)
q3p = df[col_p].quantile(0.75)
iqrp = q3p - q1p
df = df[(df[col_p] >= q1p - 1.5 * iqrp) & (df[col_p] <= q3p + 1.5 * iqrp)]
print(df[col_p].describe())

In [ ]:
# Visualización con histograma
plt.figure(figsize=(8, 6))
sns.histplot(df['Manganeso'], kde=True, bins=50)
plt.title('Distribución de Manganeso después de aplicar Logaritmo Natural')
plt.xlabel('Valor de Manganeso (log)')
plt.ylabel('Frecuencia')
plt.show()

<p> El Manganeso original tenía muchos valores muy pequeños. Después del logaritmo, la distribución ahora tiene dos picos: uno para los valores originales casi cero y otro para valores un poco más grandes. No se volvió normal, sino que muestra dos grupos distintos. Hay una gran cantidad de muestras con niveles de Manganeso casi indetectables, y otro grupo (más pequeño) con niveles de Manganeso que son significativamente mayores que cero </p>

<b style='font-size:22px'> Análisis de la columna: Turbidez </b>

In [ ]:
df['Turbidez'].describe()

<p>La variable Turbidez muestra una distribución sesgada, con valores mayoritariamente bajos y algunos extremos altos (mínimo: 9.18e-14, máximo: 19.29). Se aplica una transformación logarítmica para reducir la asimetría, facilitar su análisis y representar mejor su comportamiento en el modelo.:</p>

In [ ]:
df['Turbidez'] = df['Turbidez'] + 1e-10  # evitar log(0)
# Aplicamos el logaritmo natural
df['Turbidez'] = np.log(df['Turbidez'])

In [ ]:
#Eliminamos los Outliers:
col_p = 'Turbidez'
q1p = df[col_p].quantile(0.25)
q3p = df[col_p].quantile(0.75)
iqrp = q3p - q1p
df = df[(df[col_p] >= q1p - 1.5 * iqrp) & (df[col_p] <= q3p + 1.5 * iqrp)]
print(df[col_p].describe())

In [ ]:
# Visualización con histograma y KDE
plt.figure(figsize=(8, 6))
sns.histplot(df['Turbidez'], kde=True, bins=50)
plt.title('Distribución de Turbidez después de aplicar Logaritmo Natural')
plt.xlabel('Valor de Turbidez (log)')
plt.ylabel('Frecuencia')
plt.show()

<p>La transformación logarítmica de la columna 'Turbidez' ha comprimido el rango de valores y reducido la fuerte asimetría positiva original, concentrando la mayoría de las observaciones en valores logarítmicos bajos. Sin embargo, la distribución sigue mostrando una alta frecuencia en valores transformados cercanos a cero, indicando que la mayoría de las lecturas originales de turbidez eran muy bajas, con una cola que se extiende hacia valores logarítmicos más altos correspondientes a los picos de turbidez.</p>

<b style='font-size:22px'> Análisis de la columna: Cobre </b>

In [ ]:
df['Cobre'].describe()

<p>La variable Cobre muestra una distribución sesgada, con valores mayoritariamente bajos y algunos extremos altos (mínimo: 5.41e-08, máximo: 11.39). Se aplica una transformación logarítmica para reducir la asimetría, facilitar su análisis y representar mejor su comportamiento en el modelo.</p>

In [ ]:
df['Cobre'] = df['Cobre'] + 1e-10  # evitar log(0)

# Aplicamos el logaritmo natural
df['Cobre'] = np.log(df['Cobre'])
print(df['Cobre'].describe())

In [ ]:
#Eliminamos los Outliers:
col_p = 'Cobre'
q1p = df[col_p].quantile(0.25)
q3p = df[col_p].quantile(0.75)
iqrp = q3p - q1p
df = df[(df[col_p] >= q1p - 1.5 * iqrp) & (df[col_p] <= q3p + 1.5 * iqrp)]
print(df[col_p].describe())

In [ ]:
# Visualización con histograma
plt.figure(figsize=(8, 6))
sns.histplot(df['Cobre'], kde=True, bins=50)
plt.title('Distribución de Cobre después de aplicar Logaritmo Natural')
plt.xlabel('Valor de Cobre (log)')
plt.ylabel('Frecuencia')
plt.show()

<p>El gráfico resultante muestra cómo la transformación logarítmica mejora la simetría de la distribución de los datos de Cobre, acercándola a una distribuición más normal. </p>

<b style="font-size:22px">Análisis de columnas: Fluoruro, Olor, Sulfatos:</b>

In [ ]:
columnas = ['Fluoruro','Olor','Sulfatos']

In [ ]:
df[columnas].describe()

In [ ]:
# Grafico las columnas:'Fluoruro','Olor','Sulfatos':
fig, axes = plt.subplots(1, len(columnas), figsize=(5 * len(columnas), 4))

for i, col in enumerate(columnas):
    sns.histplot(df[col], kde=True, ax=axes[i], color='skyblue')
    axes[i].set_title(f'Distribución de {col}')
    axes[i].set_xlabel(col)
    axes[i].set_ylabel('Frecuencia')

plt.tight_layout()
plt.show()

In [ ]:
# Aplicamos IQR a las columnas a análizar:
cols = ['Fluoruro','Olor','Sulfatos']
for col in cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    df = df[(df[col] >= Q1 - 1.5 * IQR) & (df[col] <= Q3 + 1.5 * IQR)]
    df[cols].describe()

In [ ]:
# Grafico de las columnas:'Fluoruro','Olor','Sulfatos':
fig, axes = plt.subplots(1, len(columnas), figsize=(5 * len(columnas), 4))

for i, col in enumerate(columnas):
    sns.histplot(df[col], kde=True, ax=axes[i], color='skyblue')
    axes[i].set_title(f'Distribución de {col}')
    axes[i].set_xlabel(col)
    axes[i].set_ylabel('Frecuencia')

plt.tight_layout()
plt.show()

In [ ]:
df.isna().sum()

<b style="font-size:22px">Análisis de columnas: Conductividad, Cloro residual libre, Sólidos disueltos totales:</b>

In [ ]:
columnas2 = ['Conductividad','Cloro residual libre','Sólidos disueltos totales']

In [ ]:
df[columnas2].describe()

In [ ]:
#IQR
cols2 = ['Conductividad', 'Cloro residual libre', 'Sólidos disueltos totales']
for col in cols2:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    df = df[(df[col] >= Q1 - 1.5 * IQR) & (df[col] <= Q3 + 1.5 * IQR)]

In [ ]:
#Gráficamos las columnas: 'Conductividad','Cloro residual libre','Sólidos disueltos totales'
fig, axes = plt.subplots(1, len(columnas2), figsize=(5 * len(columnas), 4))

for i, col in enumerate(columnas2):
    sns.histplot(df[col], kde=True, ax=axes[i], color='skyblue')
    axes[i].set_title(f'Distribución de {col}')
    axes[i].set_xlabel(col)
    axes[i].set_ylabel('Frecuencia')

plt.tight_layout()
plt.show()

<b style="font-size:22px">Análisis de columna: Temperatura del agua</b>

In [ ]:
# Observamos la distribuición de la columna Temp. del Agua:
plt.figure(figsize=(10, 6))
sns.histplot(df['Temperatura del agua'], kde=True)
plt.title('Temperatura del agua')
plt.xlabel('Valor')
plt.ylabel('Frecuencia')
plt.show()

In [ ]:
df['Temperatura del agua'].describe()

In [ ]:
#Eliminamos los Outliers:

col9 = 'Temperatura del agua'  # ejemplo
Q1 = df[col9].quantile(0.25)
Q3 = df[col9].quantile(0.75)
IQR = Q3 - Q1
df = df[(df[col9] >= Q1 - 1.5 * IQR) & (df[col9] <= Q3 + 1.5 * IQR)]
df['Temperatura del agua'].isna().sum() #Analizo si hay valor nulos, me da 0.
df['Temperatura del agua'].describe()

In [ ]:
# Volvemos a graficar
plt.figure()
sns.histplot(df['Temperatura del agua'], kde=True)
plt.title('Temperatura del agua')
plt.xlabel('Valor')
plt.ylabel('Frecuencia')
plt.show()

<p>Después de eliminar los valores nulos y los outliers, los datos de las temperaturas muestran una reducción en la cantidad de registros, con una disminución tanto en la media como en la desviación estándar. La temperatura del agua experimentó una baja en la media, pasando de 19.13 a 17.58, lo que indica que los valores extremos estaban influyendo en el promedio. También se observó una notable reducción en el valor máximo, que bajó de 210.82 a 42.51, reflejando la eliminación de outliers muy altos.</p>

<b style="font-size:22px">Análisis de columna: Temperatura del aire</b>

In [ ]:
# Observamos la distribuición de la columna Temp. del Aire:
plt.figure(figsize=(10, 6))
sns.histplot(df['Temperatura del aire'], kde=True)
plt.title('Temperatura del aire')
plt.xlabel('Valor')
plt.ylabel('Frecuencia')
plt.show()

In [ ]:
df['Temperatura del aire'].describe()

In [ ]:
#Eliminamos los Outliers:

col10 = 'Temperatura del aire'  # ejemplo
Q1 = df[col10].quantile(0.25)
Q3 = df[col10].quantile(0.75)
IQR = Q3 - Q1
df = df[(df[col10] >= Q1 - 1.5 * IQR) & (df[col10] <= Q3 + 1.5 * IQR)]
df['Temperatura del aire'].isna().sum() #Analizo si hay valor nulos, me da 0.
df['Temperatura del aire'].describe()

In [ ]:
# Volvemos a graficar
plt.figure()
sns.histplot(df['Temperatura del aire'], kde=True)
plt.title('Temperatura del aire')
plt.xlabel('Valor')
plt.ylabel('Frecuencia')
plt.show()

In [ ]:
df.isna().sum()

<p>En cuanto a la temperatura del aire, la media prácticamente se mantuvo igual, pero el máximo también disminuyó de 140.88 a 108.70, sugiriendo que los valores extremos fueron eliminados.</p>

<h1>Sección categorica:</h1>

<b style="font-size:22px">Análisis de columna: Meses</b>

In [ ]:
print(df['Mes'].unique())
total_meses = df['Mes'].count()
print("Total de registros con Mes válido:", total_meses)

In [ ]:
# Elimino los valores nulos
df = df[df['Mes'].notna()]
# Ordeno los meses
orden_meses = ['January', 'February', 'March', 'April', 'May', 'June',
               'July', 'August', 'September', 'October', 'November', 'December']

In [ ]:
conteo_meses = df['Mes'].value_counts().reindex(orden_meses)
plt.figure(figsize=(10,5))
sns.barplot(x=conteo_meses.index, y=conteo_meses.values, palette="viridis")
plt.title("Distribución por Mes")
plt.xlabel("Mes")
plt.ylabel("Frecuencia")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

<p>Convertimos las variables categóricas en formato numérico para poder usarlas en modelos. (Encoding) Se aplica Label Encoding con orden cronológico ya que este es útil para conservar el orden de los meses y usarlo en modelos que consideren relaciones ordinales</p>

In [ ]:
#Este código toma la columna categórica 'Mes', define un orden específico para los meses y luego crea una nueva columna numérica
#'Mes_encoded' que representa los meses con números basados en ese orden. Esto prepara los datos para su uso en modelos de aprendizaje
# automático que requieren entrada numérica.

df['Mes'] = pd.Categorical(df['Mes'], categories=orden_meses, ordered=True)
df['Mes_encoded'] = df['Mes'].cat.codes

df[['Mes', 'Mes_encoded']].head()

<b style="font-size:22px">Análisis de columna: Color</b>

<p>La variable Color representa la apariencia visual del agua y puede estar relacionada con la presencia de contaminantes orgánicos o inorgánicos. Dado que estas categorías tienen una progresión lógica (desde "Colorless" hasta "Yellow"), se convierte a una variable categórica ordinal con el siguiente orden:

'Colorless' < 'Near Colorless' < 'Faint Yellow' < 'Light Yellow' < 'Yellow'. Este orden refleja un aumento en la intensidad del color, útil para modelos que aprovechan relaciones ordinales.</p>

In [ ]:
print(df['Color'].unique())
total_meses = df['Color'].count()
print("Total de registros con Mes válido:", total_meses)

In [ ]:
# Elimino los valores nulos
df = df[df['Color'].notna()]
# Este orden sigue la escala estándar de color del agua tipo Pt-Co (Platinum-Cobalt), donde un número bajo indica agua más clara.
orden_color = ['Colorless', 'Near Colorless', 'Faint Yellow', 'Light Yellow', 'Yellow']

In [ ]:
conteo_color = df['Color'].value_counts().reindex(orden_color)
plt.figure(figsize=(10,5))
sns.barplot(x=conteo_color.index, y=conteo_color.values, palette="viridis")
plt.title("Distribución por Color")
plt.xlabel("Color")
plt.ylabel("Frecuencia")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

<p>Convertimos las variables categóricas en formato numérico para poder usarlas en modelos. (Encoding) Se aplica Label Encoding con orden cronológico ya que este es útil para conservar el orden de los meses y usarlo en modelos que consideren relaciones ordinales </p>

In [ ]:
# Este código convierte la columna categórica 'Color' en una variable ordinal,
# definiendo un orden lógico basado en la intensidad del color del agua,
# desde 'Colorless' (agua más clara) hasta 'Yellow' (más intensa).
# Luego, se crea una nueva columna 'Color_encoded' con valores numéricos (0 a 4)
# que permiten utilizar esta variable en modelos de aprendizaje automático que requieren datos numéricos.
df['Color'] = pd.Categorical(df['Color'], categories=orden_color, ordered=True)
df['Color_encoded'] = df['Color'].cat.codes

df[['Color', 'Color_encoded']].head()

<b style="font-size:22px">Análisis de columna: Fuente</b>

In [ ]:
print(df['Fuente'].unique())
total_fuentes = df['Fuente'].count()
print("Total de registros con Fuente son:", total_fuentes)

In [ ]:
# Elimino los valores nulos
df = df[df['Fuente'].notna()]

In [ ]:
conteo_fuentes = df['Fuente'].value_counts()
plt.figure(figsize=(10,5))
sns.barplot(x=conteo_fuentes.index, y=conteo_fuentes.values, palette="viridis")
plt.title("Distribución por Fuentes")
plt.xlabel("Fuente")
plt.ylabel("Frecuencia")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# En el gráfico se visualizan casi iguales, confirmo con esto
conteo_fuentes = df['Fuente'].value_counts()
print(conteo_fuentes)
# Hay diferencias entre estos por lo tanto mantengo las Fuentes

In [ ]:
# La variable 'Fuente' representa la procedencia del agua (ej. río, pozo, lago, etc.).
# Dado que estas categorías no tienen un orden lógico intrínseco, se consideran variables categóricas nominales.
# Para su uso en modelos de machine learning, se recomienda aplicar One-Hot Encoding.
df_fuente_encoded = pd.get_dummies(df['Fuente'], prefix='Fuente')
df = pd.concat([df, df_fuente_encoded], axis=1)

In [ ]:
# Veo si estan todas las columnas
print(df.columns[df.columns.str.startswith('Fuente_')])

In [ ]:
# Verifico que el encoding esté bien hecho : Coincide con el valor original de Fuente.
df[['Fuente', 'Fuente_Reservoir', 'Fuente_Spring', 'Fuente_River', 'Fuente_Stream',
    'Fuente_Aquifer', 'Fuente_Lake', 'Fuente_Ground', 'Fuente_Well']].head(10)

In [ ]:
# Chequeo la suma total : da OK al original
print(df[['Fuente_Reservoir', 'Fuente_Spring', 'Fuente_River', 'Fuente_Stream',
          'Fuente_Aquifer', 'Fuente_Lake', 'Fuente_Ground', 'Fuente_Well']].sum())

In [ ]:
df.columns

In [ ]:
# Elimino las columnas que no son relevantes para el objetivo del modelo: 'Hora del día', 'Día'

In [ ]:
df = df.drop(columns=['Día', 'Hora del día'])

In [ ]:
df.columns

In [ ]:
df.dtypes

<b style="font-size:22px">Estandarización de columnas solo de variables numéricas:</b>

In [ ]:
scaler = StandardScaler()

In [ ]:
columnas_numericas = df.select_dtypes(include=['float64', 'int64']).columns.drop("Objetivo")
df[columnas_numericas] = scaler.fit_transform(df[columnas_numericas])
print(df[columnas_numericas].describe())

In [ ]:
df['Objetivo'] # La excluímos de la transformación ya que debe ser un entero.

<b font-size:26px>🧪 Conclusión Final: <br>
Se aplicó una transformación logarítmica a las variables con notación científica para reducir su sesgo y valores extremos. Posteriormente, se estandarizaron todas las variables numéricas para que tengan una media de 0 y una desviación estándar de 1. Esta estandarización es crucial al utilizar modelos de clasificación, ya que garantiza que todas las características contribuyan de manera equitativa al modelo.</b>

# Pre- Entrega N°3
- Trabajo sobre un modelo de aprendizaje supervisado.
- Ajuste de modelos de clasificación o regresión.
- Evaluación de los modelos.
- Optimización de hiperparámetros.

<p>Algoritmo a utilizar: Árbol de decisión, ya que hay varias columnas con distribuiciones que no son normales o estan sesgadas. </p>

In [ ]:
# Target debe ser de tipo entero, por lo tanto la excluimos de la transformación.
# Pasos a seguir:
# Elegir el algoritmo
# Entrenar el modelo
# Evaluar el modelo Métricas (Ver si hay overfitting)
# Ajustar hiperparámetros (tuning)
# Validación cruzada
# Interpretar el modelo

<b>1. Separamos variables predictoras y objetivo - Excluímos las categóricas </b>

In [ ]:
X = df.drop(columns=["Índice", "Objetivo", "Fuente", "Color", "Mes"])
y = df["Objetivo"]

<b>2. División entrenamiento / prueba </b>

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
#X_train, y_train: 80 % de los datos, para entrenar el modelo.
#X_test, y_test: 20 % de los datos, para evaluar el modelo (probar cómo funciona con datos que no “vio” durante el entrenamiento).

<b>3. Definimos el modelo y la grilla de hiperparámetros para GridSearchCV para hallar el mejor modelo :</b>

In [ ]:
from sklearn.model_selection import GridSearchCV
# Árbol con ajuste de peso para clases desbalanceadas (presta atención a la clase minoritaria)

dt = DecisionTreeClassifier(random_state=42, class_weight='balanced')

param_grid = {
    'max_depth': [5, 10, 20, None],
    'min_samples_split': [2, 5, 10],
    'criterion': ['gini', 'entropy']
}

<b> 4. Creamos y entrenamos el GridSearchCV (solo con datos de entrenamiento x_train y_train) y usamos validación cruzada:</b>

In [ ]:
# cv=5 La validación cruzada 5-fold que divide X_train en 5 partes y entrena/evalúa 5 veces por cada combinación.
grid_search = GridSearchCV(estimator=dt, param_grid=param_grid,
                           cv=5, scoring='f1', n_jobs=-1)

grid_search.fit(X_train, y_train)

<b>5. GridSearchCV se queda con la combinación de hiperparámetros que dio el mejor resultado. Los mejores modelos son:</b>

In [ ]:
print("Mejores parámetros:", grid_search.best_params_)
print("Mejor score F1:", grid_search.best_score_)
# Nos da el modelo con esos hiperparámetros que mejor rindió, en promedio, un F1 de 0.83 durante validación cruzada

In [ ]:
mejor_modelo = grid_search.best_estimator_

<b>6. Hacemos predicciones con los datos de x_test usando el mejor modelo: </b>

In [ ]:
y_pred_test = mejor_modelo.predict(X_test)

# y_pred : predicciones que el modelo hace sobre los datos de prueba, que normalmente son datos que el modelo no ha visto antes.
# Usa el mejor modelo para hacer predicciones sobre los datos de prueba (X_test).

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

<h1>Métricas: </h1>

In [ ]:
# La matriz de confusión corresponde solo a las predicciones hechas sobre el conjunto de prueba (X_test) 0.2 = 20%
# x_test son datos que el modelo nunca 'vio' antes.

In [ ]:
# Generamos la matriz de confusión para comparar las predicciones del modelo (y_pred) con las etiquetas reales (y_test).
cm = confusion_matrix(y_test, y_pred_test)
print("Matriz de confusión, aplicando foco a la clase minoritaria por desbalance:", cm)

<b>Antes de aplicar los hiperparametros:</b>
<ul>
<li>32448 Verdaderos Negativos (VN): El modelo predijo "No potable" y realmente no era potable.</li>
<li>2569 Falsos Positivos (FP): El modelo predijo "Potable" cuando en realidad no era potable </li>
<li>2747 Falsos Negativos (FN): El modelo predijo "No potable" cuando en realidad sí era potable.</li>
<li>7538 Verdaderos Positivos (VP): El modelo predijo "Potable" y realmente era potable.</li>
<ul>
<br>
<b>Luego de aplicar hiperparámetros: </b>
<ul>
<li>VN: 32107 El modelo acertó al decir que el agua no era potable.</li>
<li>FP: 2910 El modelo se equivocó: dijo que el agua era potable, pero no lo era.</li>
<li>FN: 749 El modelo se equivocó: dijo que el agua no era potable, pero sí lo era.</li>
<li>TP: 9536 El modelo acertó al decir que el agua era potable.</li>
</ul>
<br>
<i>Conclusión: El ajuste de hiperparámetros mejoró el modelo, especialmente para detectar correctamente cuándo el agua es potable, nuestro objetivo. </i>

In [ ]:
print("\nReporte de clasificación con hiperparámetros:")
print(classification_report(y_test, y_pred_test)) # Calcula esas métricas comparando las  predicciones y_pred con las etiquetas reales y_test.

In [ ]:
df.shape # se redujo porque no tiene en cuenta X = Índice", "Objetivo", "Fuente", "Color", "Mes" , en este punto.

In [ ]:
# Evaluar si hay overfitting. Probar otro algoritmo RandomForest

# ¿Por qué es valioso analizar el tipo de fuente?
*   Se podrá entender causas probables de potabilidad: Si ciertas fuentes como pozos o manantiales tienen mayor proporción de agua potable, eso ayuda a interpretar el problema más allá de solo predecir.
*   Complementa el modelo con insights útiles: En una presentación o entrega, mostrar gráficamente qué fuentes se asocian más o menos con potabilidad te da fuerza argumentativa.
*   Puede explicar parte del rendimiento del modelo: Si una fuente tiene pocos datos y mucha variabilidad, puede ser difícil de predecir.

Entonces se va a usar ese mismo conjunto de prueba (X_test), pero dividirlo por tipo de fuente de agua (río, pozo, lago, etc.) y ver cómo se comporta el modelo en cada una.

In [ ]:
# Lista de columnas que indican la fuente de agua
columnas_fuente = [col for col in df.columns if col.startswith("Fuente_")]

# Creamos un diccionario para guardar los resultados
resultados_fuente = {}

# Recorremos cada columna de fuente (cada una es True/False)
for col in columnas_fuente:
    # Filtramos el dataframe para quedarnos solo con las muestras de esa fuente
    df_filtrado = df[df[col] == True]

    # Calculamos la proporción de agua potable para esa fuente
    proporcion_potable = df_filtrado["Objetivo"].mean()
    cantidad = df_filtrado.shape[0]

    # Guardamos los resultados
    resultados_fuente[col.replace("Fuente_", "")] = {
        "Proporción potable": proporcion_potable,
        "Cantidad de muestras": cantidad
    }

    # Convertimos el diccionario a un dataframe ordenado por proporción potable
df_resultados_fuente = pd.DataFrame(resultados_fuente).T.sort_values(
    by="Proporción potable", ascending=False
)

# Mostramos el dataframe
print(df_resultados_fuente)

## Objetivo del análisis
Explorar si el origen del agua (fuente) afecta el desempeño del modelo de clasificación. En particular, buscamos ver si el modelo logra detectar correctamente el agua potable (clase 1) con buen balance entre precisión y recall para cada tipo de fuente.



In [ ]:
# Visualizamos la proporción de potabilidad por tipo de fuente
plt.figure(figsize=(10, 6))
sns.barplot(x=df_resultados_fuente.index, y=df_resultados_fuente["Proporción potable"], palette="viridis")
plt.title("Proporción de agua potable según fuente")
plt.ylabel("Proporción de agua potable")
plt.xlabel("Tipo de fuente")
plt.ylim(0, 0.25)
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import pandas as pd

# 1. Entrenar el modelo con todo el set de entrenamiento
rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=20,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)

rf_model.fit(X_train, y_train)

# 2. Lista de columnas que indican tipo de fuente
columnas_fuente = [
    'Fuente_Aquifer', 'Fuente_Ground', 'Fuente_Lake',
    'Fuente_Reservoir', 'Fuente_River', 'Fuente_Spring', 'Fuente_Stream', 'Fuente_Well'
]

# 3. Evaluar el modelo por cada tipo de fuente
for fuente in columnas_fuente:
    print(f"\n🔍 Evaluando para: {fuente.replace('Fuente_', '')}")

    # Filtramos solo las muestras de prueba que provienen de esta fuente
    filtro = X_test[X_test[fuente] == True]

    # Si no hay muestras, pasamos
    if filtro.empty:
        print("No hay muestras de esta fuente en el set de prueba.")
        continue

    # Predicciones para esta fuente
    y_real = y_test.loc[filtro.index]
    y_pred = rf_model.predict(filtro)

    # Reporte de métricas
    print(classification_report(y_real, y_pred, digits=3))

El modelo Random Forest mantiene un rendimiento constante y sólido sin importar el tipo de fuente. Todos los valores de accuracy están alrededor del 91%-93%.

El F1-score para la clase 1 (agua potable), que es nuestro objetivo principal, se mantiene alto en todas las fuentes (entre 0.839 y 0.862). Esto significa que el modelo detecta correctamente la mayoría de los casos de agua potable y con buena precisión.

El recall para clase 1 ronda el 97-98% en todas las fuentes, lo que indica que el modelo casi nunca falla en detectar agua potable.

No es necesario entrenar modelos por separado por fuente, ya que no hay un tipo de fuente que tenga un mal desempeño o que degrade el rendimiento general.

In [ ]:
importancias = rf_model.feature_importances_
nombres = X.columns

df_importancias = pd.DataFrame({"Variable": nombres, "Importancia": importancias})
df_importancias = df_importancias.sort_values("Importancia", ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x="Importancia", y="Variable", data=df_importancias.head(15))
plt.title("🎯 Importancia de variables en Random Forest")
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

# Supongamos que X es tu dataset de entrada y y es el target
X = df.drop(columns=["Índice", "Objetivo", "Fuente", "Color", "Mes"])
y = df["Objetivo"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Support Vector Machine
Es un algoritmo supervisado usado tanto para clasificación como regresión. La idea central es encontrar el hiperplano que mejor separa las clases en el espacio de características. Es especialmente útil cuando hay claras fronteras entre clases y no muchos outliers. Para problemas no lineales, SVM puede transformar el espacio usando kernels (como RBF o polinómico) para lograr una separación más clara.

In [ ]:
# Usar solo el 20% del entrenamiento para probar SVM más rápido
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

# Define the pipeline_svm here
pipeline_svm = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', SVC(kernel='linear', C=1, class_weight='balanced', random_state=42))
])


X_train_svm, _, y_train_svm, _ = train_test_split(
    X_train, y_train, train_size=0.2, stratify=y_train, random_state=42
)

# Volvemos a entrenar el pipeline con menos datos
pipeline_svm.fit(X_train_svm, y_train_svm)

# Predicción sobre el test completo
y_pred_svm = pipeline_svm.predict(X_test)

# Reporte
print("🔍 Reporte de clasificación para SVM con 20% de los datos:")
print(classification_report(y_test, y_pred_svm, digits=3))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)


In [ ]:
cm = confusion_matrix(y_test, y_pred_svm)

plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=["No potable", "Potable"],
            yticklabels=["No potable", "Potable"])
plt.xlabel("Predicción")
plt.ylabel("Valor real")
plt.title("Matriz de Confusión - SVM (20% del entrenamiento)")
plt.tight_layout()
plt.show()


**Interpretación de las métricas**

✅ Precisión
Clase 0 (No potable): 0.908 → El modelo es muy preciso al decir que el agua no es potable, es decir, cuando predice "no potable", casi siempre acierta.

Clase 1 (Potable): 0.484 → La precisión es baja. Significa que cuando el modelo dice que el agua es potable, se equivoca más de la mitad de las veces.

✅ Exhaustividad (Recall)
Clase 0: 0.770 → Detecta el 77% de los verdaderos casos de "no potable".

Clase 1: 0.734 → Este valor es bueno, significa que detecta correctamente el 73% de los casos donde el agua sí es potable (recuerda que esta es la clase minoritaria).

✅ F1-Score
Clase 0: 0.833 → Buen equilibrio entre precisión y recall para detectar agua no potable.

Clase 1: 0.583 → Es moderado, porque aunque detecta bien los potables, se equivoca mucho en la predicción.

## 📌 1️⃣ Comparar métricas clave de los modelos

In [ ]:
# 📝 Qué hace: crea una tabla clara para comparar rendimiento de los tres modelos con las métricas más usadas.
# Supongamos que ya calculaste estas métricas antes para cada modelo:
# (Reemplazá con los valores reales que obtuviste en tu notebook)

accuracy_dt = 0.92
precision_dt = 0.75
recall_dt = 0.97
f1_dt = 0.84

accuracy_rf = 0.92
precision_rf = 0.75
recall_rf = 0.97
f1_rf = 0.85

accuracy_svm = 0.91
precision_svm = 0.70
recall_svm = 0.99
f1_svm = 0.82

# Creamos un DataFrame para comparar
import pandas as pd

resultados = pd.DataFrame({
    "Modelo": ["Decision Tree", "Random Forest", "SVM"],
    "Accuracy": [accuracy_dt, accuracy_rf, accuracy_svm],
    "Precision": [precision_dt, precision_rf, precision_svm],
    "Recall": [recall_dt, recall_rf, recall_svm],
    "F1": [f1_dt, f1_rf, f1_svm]
})

# Mostramos la tabla comparativa
print("🔎 Comparación de métricas:")
print(resultados)


## 📌 2️⃣ Graficar matriz de confusión como heatmap


In [ ]:
# Qué hace: dibuja las matrices de confusión para visualizar de forma clara aciertos y errores de cada modelo.
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Función para graficar matriz de confusión
def plot_cm(y_true, y_pred, titulo):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(5,4))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
    plt.title(titulo)
    plt.xlabel('Predicho')
    plt.ylabel('Real')
    plt.show()

# Usá esto para cada modelo (reemplazá y_pred_xxx con tus predicciones)
# plot_cm(y_test, y_pred_dt, "Matriz de Confusión - Decision Tree")
# plot_cm(y_test, y_pred_rf, "Matriz de Confusión - Random Forest")
# plot_cm(y_test, y_pred_svm, "Matriz de Confusión - SVM")


In [ ]:
# 3️⃣ Ver importancia de las variables (para árboles)
# Para Random Forest o Decision Tree
importances = rf_model.feature_importances_
features = X.columns
importancia_df = pd.DataFrame({"Variable": features, "Importancia": importances})
importancia_df = importancia_df.sort_values(by="Importancia", ascending=False)

# Mostrar top 10
print("🔎 Top 10 variables más importantes según Random Forest:")
print(importancia_df.head(10))

# (Opcional) Gráfico
sns.barplot(x="Importancia", y="Variable", data=importancia_df.head(10))
plt.title("Importancia de variables - Random Forest")
plt.show()



➡️ Se compararon tres modelos: Decision Tree, Random Forest y SVM.
➡️ Random Forest tuvo el mejor equilibrio entre precision y recall, con un f1-score más alto.
➡️ SVM mostró un recall muy alto pero menor precision, lo que indica que predice casi todos los potables pero con más falsos positivos.
➡️ Las variables más importantes fueron: [acá listás las top 3 de la tabla].
➡️ A futuro se podría:
- Probar XGBoost o LightGBM.
- Aplicar reducción de dimensionalidad (PCA).
- Incluir nuevas variables contextuales.